In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from PIL import Image
from tensorflow.keras.preprocessing import image as keras_image
from colabcode import ColabCode

In [2]:

# Mount Google Drive to access the dataset
mount_drive('/content/drive')  # Update path accordingly

NameError: name 'mount_drive' is not defined

In [ ]:
# Define paths to your dataset folders
train_path = '/content/drive/MyDrive/Garbage/Train'
test_path = '/content/drive/MyDrive/Garbage/Test'

In [ ]:
# Preprocess and augment the images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
#classes = list(validation_generator.class_indices.keys())

NameError: name 'test_datagen' is not defined

In [ ]:
# Load VGG16 model without top layers
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Create a new model
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))  # 6 classes


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1.0e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# View structure of the model
from keras.callbacks import ModelCheckpoint, EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                        

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=50,  # Update the number of epochs as needed
    verbose=1,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/50
83/83 [==============================] - ETA: 0s - loss: 0.8932 - accuracy: 0.6321 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


83/83 [==============================] - 2699s 33s/step - loss: 0.8932 - accuracy: 0.6321 - val_loss: 0.7741 - val_accuracy: 0.6039
Epoch 2/50
 6/83 [=>............................] - ETA: 27:53 - loss: 0.7571 - accuracy: 0.7240

In [ ]:
# Assuming you have loaded your test data and model as previously described
# Load and preprocess test data
#test_data = ...  # Load and preprocess your test data similar to the training data
classes = [ 'plastic', 'paper','metal']
# Make predictions using the trained model on test data
predictions1 = model.predict(validation_generator)



# Decode the predicted classes (assuming 'classes' is a list of class labels)
predicted_labels1 = [classes[np.argmax(pred)] for pred in predictions1]


# Obtain true labels for test data
true_labels1 = validation_generator.classes  # Assuming you used ImageDataGenerator for test data


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Convert class indices to class names (if true_labels are integers)
#true_labels1 = [classes[label] for label in true_labels1]
true_labels1 = [classes[label_index] for label_index in validation_generator.classes]

# Obtain true labels for the training data
# Or, convert class names to class indices (if predicted_labels are strings)
#predicted_labels1 = [classes[label] for label in predicted_labels1]

# Calculate confusion matrix
conf_matrix_test = confusion_matrix(true_labels1, predicted_labels1)

# Print confusion matrix
print("Confusion Matrix for test:")
print(conf_matrix_test)

# Calculate classification metrics
accuracy1 = accuracy_score(true_labels1, predicted_labels1)
classification_rep1 = classification_report(true_labels1, predicted_labels1)

# Print accuracy and classification report
print(f"\nAccuracy: {accuracy1 * 100:.2f}%")
print("\nClassification Report:")
print(classification_rep1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_test, annot=True, cmap='Blues', fmt='g',xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted labels for test')
plt.ylabel('True labels for test')
plt.title('Confusion Matrix for test')
plt.show()

In [ ]:
#predictions2 = model.predict(train_generator)
#predicted_labels2 = [classes[np.argmax(pred)] for pred in predictions2]
#true_labels2 = train_generator.classes

In [ ]:
# Make predictions on the training data
predictions_train = model.predict(train_generator)

# Decode predictions into class labels
predicted_labels_train = [classes[np.argmax(pred)] for pred in predictions_train]

# Obtain true labels for the training data
# Load specific class labels into true_labels_train
specified_classes = ['plastic', 'paper', 'metal']  # Replace with your specified classes
true_labels_train = [classes[label_index] for label_index in train_generator.classes]
#true_labels_train = [classes[label_index] for label_index in train_generator.classes if classes[label_index] in specified_classes]

# Generate confusion matrix
conf_matrix_train = confusion_matrix(true_labels_train, predicted_labels_train, labels=specified_classes)

print("Confusion Matrix for training data:")
print(conf_matrix_train)
accuracy2 = accuracy_score(true_labels_train, predicted_labels_train)
classification_rep2 = classification_report(true_labels_train, predicted_labels_train, target_names=specified_classes)

positive_classes = ['plastic', 'paper']
negative_class = 'metal'

# Convert 3x3 matrix to 2x2 matrix
conf_matrix_train1 = confusion_matrix(true_labels_train, predicted_labels_train, labels=[negative_class] + positive_classes)

# Calculate True Positive (TP), False Positive (FP), True Negative (TN), False Negative (FN)
TN = conf_matrix_train[0, 0]
FP = conf_matrix_train[0, 1]
FN = conf_matrix_train[1, 0]
TP = conf_matrix_train[1, 1]

# Calculate accuracy
accuracy8 = (TP + TN) / (TP + TN + FP + FN)

print("Confusion Matrix for training data (2x2):")
print(conf_matrix_train1)
print("\nAccuracy:", accuracy8)
# Print accuracy and classification report
print(f"\nAccuracy: {accuracy2 * 100:.2f}%")
print("\nClassification Report:")
print(classification_rep2)

In [ ]:
# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_train, annot=True, cmap='Blues', fmt='g', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted labels for train')
plt.ylabel('True labels for train')
plt.title('Confusion Matrix for train')
plt.show()

In [ ]:

# Assuming you have obtained true_labels and predicted_labels as described earlier
# Calculate classification report
class_rep = classification_report(true_labels1, predicted_labels1, target_names=classes, output_dict=True)

# Extract precision, recall, and F1-score from the classification report
precision = [class_rep[label]['precision'] for label in classes]
recall = [class_rep[label]['recall'] for label in classes]
f1_score = [class_rep[label]['f1-score'] for label in classes]

# Plotting precision, recall, and F1-score
plt.figure(figsize=(10, 6))

plt.bar(classes, precision, alpha=0.5, color='blue', label='Precision')
plt.bar(classes, recall, alpha=0.5, color='red', label='Recall')
plt.bar(classes, f1_score, alpha=0.5, color='green', label='F1-score')

plt.xlabel('Classes')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1-score by Class for validation')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
class_rep2 = classification_report(true_labels_train, predicted_labels_train, target_names=classes, output_dict=True)

# Extract precision, recall, and F1-score from the classification report
precision2 = [class_rep[label]['precision'] for label in classes]
recall2 = [class_rep[label]['recall'] for label in classes]
f1_score2 = [class_rep[label]['f1-score'] for label in classes]

# Plotting precision, recall, and F1-score
plt.figure(figsize=(10, 6))

plt.bar(classes, precision2, alpha=0.5, color='blue', label='Precision')
plt.bar(classes, recall2, alpha=0.5, color='red', label='Recall')
plt.bar(classes, f1_score2, alpha=0.5, color='green', label='F1-score')

plt.xlabel('Classes')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1-score by Class for train')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:

import matplotlib.pyplot as plt
# Plotting the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plotting the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
model.save('/content/drive/MyDrive/Garbage/model.h5')

In [ ]:
# Evaluate the model
loss99, accuracy99 = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy99 * 100:.2f}%")


In [ ]:
# Evaluate the model
loss55, accuracy55 = model.evaluate(train_generator)
print(f"Training Accuracy: {accuracy55 * 100:.2f}%")


In [ ]:


def load_image(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img = keras_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalization
    return img


In [ ]:
# Make predictions
new_image = load_image('/content/drive/MyDrive/Garbage/photos/Bill.jpg')

prediction = model.predict(new_image)
classes = ['metal', 'paper','plastic']
predicted_class_index = np.argmax(prediction)
predicted_class = classes[predicted_class_index]
test_input = new_image.reshape((224,224,3))
plt.imshow(test_input)

#import cv2
#new_image = cv2.imread('/content/drive/MyDrive/Garbage/Test/Bill.jpg')
#plt.imshow(new_image)
#new_image.shape


# Print the predicted class
print(f"The waste in the image is classified as: {predicted_class}")